In [388]:
import pandas as pd
from pathlib import Path
import itertools
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import numpy as np
import pandas as pd
from pathlib import Path

# TODO
- [ ] Add additional documentation 
- [ ] split multiple abbreviations into separate columns 
- [ ] Identify top k abbreviation candidates for 

# Notebook aim 
- identify abbreviations which might be useful to disambiguate using cv in maps 

### Read abbreviations

In [389]:
os_abbreviation = pd.read_excel("data/OS_Abbreviations.xlsx")
os_abbreviation.head(10)

,label,text
0,A,"Alley, Approach, Arcade, Avenue"
1,A,Area
2,A,Telephone Call Box (AA) (small scales)
3,AA,AA call box
4,A/D,Asphalt Disc
5,AA,Automobile Association
6,AAT,Analytical Aerial Triangulation
7,Aber,Aberdeenshire
8,ABUT,Abutment
9,Acad,Academy


# Sort abbreviation by frequency

In [390]:
os_abbreviation['freq'] = os_abbreviation.groupby('label')['label'].transform('count')

In [391]:
os_abbreviation.sort_values('freq', ascending=False).head(50)

,label,text,freq
615,P,Post Office (small scales),10.0
614,P,Post,10.0
613,P,Pole,10.0
612,P,Pillar,10.0
611,P,Passenger,10.0
617,P,Pylon,10.0
610,P,"Passage, Path, Plate, Pond, Post",10.0
609,P,"Parade, Pass, Passage, Path, Place, Precinct, ...",10.0
608,P,"Paling, Passage, Pavement, Pillar, Plug, Pool,...",10.0
616,P,Pump,10.0


In [392]:
os_abbreviation.sort_values('freq', ascending=False).head(30)

,label,text,freq
615,P,Post Office (small scales),10.0
614,P,Post,10.0
613,P,Pole,10.0
612,P,Pillar,10.0
611,P,Passenger,10.0
617,P,Pylon,10.0
610,P,"Passage, Path, Plate, Pond, Post",10.0
609,P,"Parade, Pass, Passage, Path, Place, Precinct, ...",10.0
608,P,"Paling, Passage, Pavement, Pillar, Plug, Pool,...",10.0
616,P,Pump,10.0


In [393]:
os_abbreviation.set_index("label", inplace=True)

# Look at 'w' abbreviation 

In [394]:
os_abbreviation.loc['W']

,text,freq
label,,
W,"Walk, Way, Wharf",7.0
W,"Wall, Weir",7.0
W,"Water, Wood",7.0
W,Watershed,7.0
W,Well,7.0
W,Well,7.0
W,West,7.0


In [395]:
c_abbreviations = os_abbreviation.loc['C']['text'].to_list()
c_abbreviations = [c for split_c in c_abbreviations for c in split_c.split(',')]
len(c_abbreviations)

30

# GB1900 

In [396]:
with open("data/gb1900_gazetteer_complete_july_2018.csv", encoding='UTF-16') as f:
    df = pd.read_csv(f)

/usr/local/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [397]:
df.head(20)

,pin_id,final_text,nation,local_authority,parish,osgb_east,osgb_north,latitude,longitude,notes
0,52b34d8b695fe90005004e1e,F. P.,Wales,Powys,Llansilin,320836.712742,327820.182715,52.842050,-3.176744,NaN
1,5800a6b92c66dcab3d061796,Parly. & Munl Boro. By.,England,City of London,NaN,531794.825962,180705.741898,51.509918,-0.102246,NaN
2,5800a6782c66dcab3d061786,S. Ps.,England,City of London,NaN,531736.217116,180725.027730,51.510105,-0.103083,NaN
3,57f684f42c66dcab3d01c0dd,Southwark Bridge Stairs,England,City of London,NaN,532199.584123,180696.934434,51.509744,-0.096420,NaN
4,57f685002c66dcab3d01c0e9,St. Paul's Pier,England,City of London,NaN,531987.486097,180745.664556,51.510232,-0.099456,NaN
5,59eb8cae2c66dc79120026d6,Pier,England,City of London,NaN,531647.214047,180767.304005,51.510505,-0.104349,NaN
6,57f00e9e2c66dca322010ab4,Landing Place,England,Southwark,NaN,531537.695456,180566.073891,51.508722,-0.106001,NaN
7,57f3c17e2c66dcab3d00984c,Landing Stages,England,Lambeth,NaN,531307.416393,180540.056952,51.508542,-0.109327,NaN
8,57e526222c66dc43f4000dcd,New Scotland Yard,England,Westminster,NaN,530369.099820,179827.783170,51.502358,-0.123103,NaN
9,57e55bd82c66dc43f4001ee8,ST. THOMAS'S HOSPITAL,England,Westminster,NaN,530421.850835,179456.055706,51.499006,-0.122480,NaN


In [398]:
min_counts = 5000
xy2plot = df.groupby("final_text").count().sort_values("pin_id", ascending=False)["pin_id"]
xy2plot_filtered = xy2plot[xy2plot > min_counts]
print("Total number of unique text strings: {}".format(len(xy2plot)))
print("Number of unique text strings (after filtering): {}".format(len(xy2plot_filtered)))

Total number of unique text strings: 692648
Number of unique text strings (after filtering): 30


In [399]:
# list text/number of xy2plot_filtered
first_index = 0
last_index = 50
xy2plot_filtered[first_index:last_index]

final_text
F. P.                306583
W                    190979
P                    115877
F. B.                 74514
Spring                46876
S. P.                 39571
G. P                  38842
Old Quarry            24673
B. S.                 22363
M. S.                 22161
Ford                  21926
M. P                  21241
Quarry                15042
B. R.                 14891
School                14351
L. B                  14279
Stone                 13625
Lodge                 12661
P. O.                  9030
S. Ps.                 8841
Weir                   8840
B. P.                  8528
Smithy                 7330
Chap.                  7177
Gravel Pit             7029
Union & R. D. By.      6887
Ws                     6868
Old Shaft              6436
Sch.                   6422
Smy.                   6189
Name: pin_id, dtype: int64

In [400]:
# read OS_Abbreviations.xlsx file which contains mapping between label and text
os_abbreviation = pd.read_excel("data/OS_Abbreviations.xlsx")

search_string = "post office"
list_of_selected_text = os_abbreviation[os_abbreviation["text"].str.contains(search_string, 
                                                                             case=False, 
                                                                             na=False)].label.to_list()
list_of_selected_text.extend([search_string])
print("Here is the list of selected texts that contain {}: {}".format(search_string, list_of_selected_text))

for itext in list_of_selected_text:
    print(10*"=" + " {}".format(itext))
    df_filtered = df[df["final_text"].str.contains(itext, case=False, na=False)]
    print(df_filtered.groupby("final_text").count().sort_values("pin_id", ascending=False)["pin_id"][0:40])

Here is the list of selected texts that contain post office: ['GPO', 'HPO', 'P', 'PO', 'post office']
========== GPO
final_text
Washingpool Farm                                   4
LANGPORT                                           3
Springpool Wood                                    2
Frogpool                                           2
Curlingpond Plantation                             2
Washingpool Burn                                   2
Washingpool Plantation                             2
GPO                                                2
Shiningpool Knowe                                  1
Shiningpool Burn                                   1
Shiningpool Bridge                                 1
Shiningpool Bank                                   1
Rumblingpots                                       1
Rubbingpost Hill                                   1
Reservoir Langport R. D. Council W. W.             1
Slingpool Walk                                     1
Reservoir (Langport R. D

final_text
Post Office                             326
post office                              25
School & Post Office                      5
POST OFFICE                               5
Greenbank (Post Office)                   2
Post office                               2
Post Office Telegraph Box                 2
Post Office Lane                          2
POST OFFICE ST.                           2
Lodge & Post Office                       2
POST OFFICE ROAD                          2
Errol Post Office                         1
Post office (sorting)                     1
Polmood Post Office                       1
Church House (Post Office)                1
Post Office Inn                           1
Post Office Knowe                         1
Bridge End (Post Office)                  1
Post Office Plantation                    1
Post Office Row                           1
Post Office St.                           1
Bogside Sta. & Post Office                1
Blackhorse Hill (Post

In [402]:
df.set_index("final_text", inplace=True)

In [405]:
len(df.loc['post office' or 'Post Office', ])

25

# Potential candidate
- post office? 326+ non abbreviated examples. Presumably in dense urban areas potential for more to be abbreviated to p. 